In [2]:
import requests
import json
import pandas
import geopandas

### Load in OA data for Leeds
- https://geoportal.statistics.gov.uk/datasets/6beafcfd9b9c4c9993a06b6b199d7e6d/explore?location=53.797284%2C-1.541437%2C11.68

In [10]:
# read in OAs (UK Wide File) within leeds bounds (Read in LAD file)
Leeds_Boundary_gdf = geopandas.read_file("Data/Boundaries/BoundaryData_Leeds/england_ltla_2022.shp")
bbox = Leeds_Boundary_gdf.bounds

# calculate total bounds & construct tuple to use when reading in a subset of a file in geopandas
min_x = bbox['minx'].min()
min_y = bbox['miny'].min()
max_x = bbox['maxx'].max()
max_y = bbox['maxy'].max()
bbox_tuple = (min_x, min_y, max_x, max_y)

# read in OAs
Leeds_OA_gdf = geopandas.read_file('Data/Boundaries/OA_Boundaries/Output_Areas_2021_EW_BGC_V2_-6055784085649114951.gpkg', bbox=bbox_tuple)

### Load in surface water suceptability dataset tiles
#### Merge together and clip to the Leeds boundary

In [8]:
def merge_floodng_files(folder, tiles, extent):
    gdfs = []
    for tile in tiles:
        file = f'{folder}/RoFSW-{tile}/RoFSW_{tile}_Extent_{extent}.shp'
        gdfs.append(geopandas.read_file(file))
    gdf = pandas.concat(gdfs)
    gdf = gdf.set_crs(27700)
    # gdf.to_crs(4326, inplace = True)
    gdf = gdf.drop_duplicates()

    return gdf

tiles = [
    'SE44',
    'SE43',
    'SE42',
    'SE34',
    'SE33',
    'SE32',
    'SE24',
    'SE23',
    'SE22',
    'SE14',
    'SE13',
    'SE12',
]

Surface_Water_Flooding_gdf = merge_floodng_files('Data/Flooding', tiles, '1in100')
Surface_Water_Flooding_gdf.to_file("Data/Processed/Leeds_RoFSW_Extent_1in100.geojson", driver="GeoJSON")

# Clip to leeds bounds
clipped_surface_water_flooding_gdf = geopandas.clip(Surface_Water_Flooding_gdf, Leeds_Boundary_gdf)
clipped_surface_water_flooding_gdf.to_file("Data/Processed/Leeds_RoFSW_Extent_1in100_Clipped.geojson", driver="GeoJSON")

### Load in OSM Building polygons

In [12]:
def get_osm_residential_buildings():
    # Overpass API endpoint
    overpass_url = "http://overpass-api.de/api/interpreter"

    # Overpass QL query to get residential buildings in Leeds
    overpass_query = """
    [out:json][timeout:60];
    area[name="Leeds"]->.searchArea;
    (
    way["building"~"^(house|residential|apartments|detached|semidetached_house|terrace|dormitory|bungalow|static_caravan)$"](area.searchArea);
    relation["building"~"^(house|residential|apartments|detached|semidetached_house|terrace|dormitory|bungalow|static_caravan)$"](area.searchArea);
    );
    out body;
    >;
    out skel qt;
    """

    # Make the request to the Overpass API
    response = requests.get(overpass_url, params={'data': overpass_query})

    # Check if the request was successful, load the data, and save to geojson file
    if response.status_code == 200:
        data = response.json()

        buildings_file_path = 'Data/Processed/Leeds_Residential_Buildings.json'

        # Save the response to a GeoJSON file
        with open(buildings_file_path, 'w') as f:
            json.dump(data, f)

        geopandas.read_file(buildings_file_path)

        
    else:
        print(f"Error: {response.status_code}")

get_osm_residential_buildings()

DriverError: 'Data/Processed/Leeds_Residential_Buildings.json' not recognized as a supported file format.

### Estimate residential population per building

In [5]:
import geopandas.tools
import pandas
import json
from shapely.geometry import Polygon
import overpy
from shapely.geometry import Polygon, MultiPolygon

# Open the JSON file and load it into a dictionary
with open('Data/Processed/Leeds_Residential_Buildings.json', 'r') as file:
    data = json.load(file)

    # Extract elements from the response
    ways = [element for element in data['elements'] if element['type'] == 'way']
    nodes = {element['id']: (element['lon'], element['lat']) for element in data['elements'] if element['type'] == 'node'}

    # Function to construct a polygon from a way
    def way_to_polygon(way, nodes):
        coords = [(nodes[node_id][0], nodes[node_id][1]) for node_id in way['nodes'] if node_id in nodes]
        if len(coords) < 3:
            return None  # Not enough points to form a polygon
        return Polygon(coords)

    # Convert ways to polygons
    polygons = [way_to_polygon(way, nodes) for way in ways if way_to_polygon(way, nodes)]

    # Create a GeoDataFrame
    df = pandas.DataFrame({'geometry': polygons})
    buildings_gdf = geopandas.GeoDataFrame(df, geometry='geometry', crs='EPSG:4326')
    buildings_gdf.to_file('Data/Processed/Leeds_OSM_Buildings.geojson', driver="GeoJSON")


In [11]:
# convert geometry into British national Grid for more accurate spatial join
buildings_gdf['geometry'] = buildings_gdf.to_crs(27700)

# Join the OA gdf to the census data at OA level - https://www.nomisweb.co.uk/sources/census_2021_bulk#:~:text=2021%20Census%20Bulk%20Data%20Download,following%20on%20row%202%20onwards.
census_population_df = pandas.read_csv('Data/Census/census2021-ts007a-oa.csv')
Leeds_OA_gdf = Leeds_OA_gdf.merge(census_population_df, left_on='OA21CD', right_on='geography')

# Spatial join the OAs to the buildings, keeping all the buildings in the join and duplicating the OA properties onto each
joined_gdf = geopandas.sjoin(buildings_gdf, Leeds_OA_gdf, how='left', op='intersects')

# Calculate the area of each building
joined_gdf['building_area'] = joined_gdf['geometry'].area

# Get the total area of buildings that intersect each OA 
# Group by the OA each building falls within, creating a new df with a column for total area of buildings within each OA
# Merge the total building areas back to the buildings gdf
oa_building_area = joined_gdf.groupby('OA21CD')['building_area'].sum().reset_index()
oa_building_area.rename(columns={'building_area': 'total_oa_building_area'}, inplace=True)
joined_gdf = joined_gdf.merge(oa_building_area, on='OA21CD', how='left')

# Add a column called building population by doing this calculation = ((Census total population of the OA / total area of buildings) * individual building area)
joined_gdf['building_population'] = ((joined_gdf['Age: Total'] / joined_gdf['total_oa_building_area']) * joined_gdf['building_area']).fillna(0)

# Name the gdf appropriately
building_population_gdf = joined_gdf

# Clip to leeds bounds
clipped_building_population_gdf = geopandas.clip(building_population_gdf, Leeds_Boundary_gdf)

# Remove unused columns
columns_to_keep = ['geometry', 'OA21CD', 'building_population']
clipped_building_population_gdf = clipped_building_population_gdf[columns_to_keep]

# Save file
clipped_building_population_gdf.to_file("Data/Processed/Leeds_Residential_Buildings_Population.geojson", driver="GeoJSON")

/var/folders/sz/n80kgmcs43b6swmgv_64_48c0000gn/T/ipykernel_9605/3021250100.py:2: UserWarning: Geometry column does not contain geometry.
  buildings_gdf['geometry'] = buildings_gdf.to_crs(27700)
/Users/alexandertrout/opt/anaconda3/envs/mac_conda_py3_10/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3490: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [12]:
# // clip buildings gdf flooding gdf keep any buoldings that intersect
building_populations_at_risk_of_flooding_gdf = geopandas.sjoin(clipped_building_population_gdf, clipped_surface_water_flooding_gdf, how="inner", op="intersects")

# // save the result to a file
building_populations_at_risk_of_flooding_gdf.to_file("building_populations_at_risk_of_flooding_gdf.geojson", driver="GeoJSON")

/Users/alexandertrout/opt/anaconda3/envs/mac_conda_py3_10/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3490: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [21]:
# // generate the values for output
total_building_pop = clipped_building_population_gdf['building_population'].sum()
total_building_pop_at_risk = building_populations_at_risk_of_flooding_gdf['building_population'].sum()
building_pop_risk_percentage = (total_building_pop_at_risk / total_building_pop) * 100
total_building_count = len(clipped_building_population_gdf)
total_building_count_at_risk = len(building_populations_at_risk_of_flooding_gdf)
total_building_risk_percentage = (total_building_count_at_risk / total_building_count) * 100
population_per_building_total = total_building_pop / total_building_count
population_per_building_at_risk = total_building_pop_at_risk / total_building_count_at_risk

# arrange the values and create the pandas dataframe
data = {
    "": ["Total", "At Risk", "Percentage"],
    "Building Count": [total_building_count, total_building_count_at_risk, total_building_risk_percentage],
    "Population Total": [total_building_pop, total_building_pop_at_risk, building_pop_risk_percentage],
    "Population Per Building": [f"{population_per_building_total:.2f}", f"{population_per_building_at_risk:.2f}", ""]
}
df = pandas.DataFrame(data)

df.to_csv('Data/Processed/building_population_risk_results_table.csv')


### Intersect building outlines with surface water suceptability layer